In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_weights: str | None
    params_classes: int
    params_freeze_base: bool   


In [6]:
from src.ccclassifier.constants import *
from src.ccclassifier.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create root artifacts directory
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.base_model.image_size,
            params_learning_rate=self.params.training.learning_rate,
            params_weights=self.params.base_model.weights,
            params_classes=self.params.base_model.num_classes,
            params_freeze_base=self.params.base_model.freeze_base,
        )

        return prepare_base_model_config

In [ ]:
#import yaml
#import urllib.request as request
#from zipfile import ZipFile

import torch
import torch.nn as nn
import torchvision.models as models
from torchinfo import summary

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = None

    def get_base_model(self):
        """Load pretrained ResNet backbone"""
        weights = None
        if self.config.params_weights:  # e.g. "IMAGENET1K_V1"
            weights = getattr(models, "ResNet18_Weights").IMAGENET1K_V1

        self.model = models.resnet18(weights=weights)

        # save just backbone weights (state_dict only)
        self.save_model(self.config.base_model_path, self.model.state_dict())
        return self.model

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, learning_rate):
        """Freeze layers + replace classifier"""
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False

        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, classes)

        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        loss_fn = nn.CrossEntropyLoss()

        return model, optimizer, loss_fn

    def update_base_model(self):
        """Prepare full model for training"""
        self.model, self.optimizer, self.loss_fn = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=self.config.params_freeze_base,
            learning_rate=self.config.params_learning_rate
        )

        # save updated model weights (best practice: state_dict)
        self.save_model(self.config.updated_base_model_path, self.model.state_dict())

    @staticmethod
    def save_model(path: Path, obj):
        """Save model weights or optimizer state"""
        path.parent.mkdir(parents=True, exist_ok=True)
        torch.save(obj, path)

In [ ]:
try:
    # Step 1: Load configuration
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()

    # Step 2: Initialize component
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)

    # Step 3: Get base model (backbone only)
    model = prepare_base_model.get_base_model()
    print(">> Base model (ResNet18) loaded and backbone weights saved.")

    # Show backbone summary
    summary(model, input_size=(1, 3, *prepare_base_model_config.params_image_size))

    # Step 4: Update with classifier head + optimizer/loss
    prepare_base_model.update_base_model()
    print(">> Full model prepared and updated weights saved.")

    # Show updated model summary
    summary(model, input_size=(1, 3, *prepare_base_model_config.params_image_size))

    #  Print trainable parameter count
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable params: {trainable_params:,} / {total_params:,} total")

except Exception as e:
    print("Error in PrepareBaseModel pipeline:", str(e))
    raise e

[2025-09-06 08:46:51,762: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-06 08:46:51,766: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-06 08:46:51,767: INFO: common: created directory at: artifacts]
[2025-09-06 08:46:51,769: INFO: common: created directory at: artifacts/prepare_base_model]
>> Base model (ResNet18) loaded and backbone weights saved.
>> Full model prepared and updated weights saved.
Trainable params: 1,539 / 11,178,051 total
